In [ ]:
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage, misc
import numpy as np
from PIL import Image
from numpy import asarray
import playsound
from playsound import playsound
import time

CONFIDENCE_THRESHOLD = 0.2
NMS_THRESHOLD = 0.2
COLORS = [(0, 255, 255), (255, 255, 0), (0, 255, 0), (255, 0, 0)]

class_names = []
with open(r'C:\Users\aksha\Downloads\coco.names.txt', "r") as f:
    class_names = [cname.strip() for cname in f.readlines()]

net = cv2.dnn.readNet(r'C:\Users\aksha\Downloads\yolov3.weights', r'C:\Users\aksha\Downloads\yolov3.cfg.txt')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

model = cv2.dnn_DetectionModel(net)
model.setInputParams(size=(416, 416), scale=1/255, swapRB=True)
i=0
count=0
time_skips = float(40000)
vc = cv2.VideoCapture(r'E:\project\PXL_20220516_122823991_Trim.mp4')
while cv2.waitKey(1) < 1:
    (grabbed, frame) = vc.read()
    if not grabbed:
        exit()
    vc.set(cv2.CAP_PROP_POS_MSEC,(count*time_skips)) 
    color=(0,0,255)
    #flag1=0
    step=10
    i=1
#     fig=plt.figure(figsize=(16,16))
#     row=10
#     col=10
    chair_list=[]
    human_count=0
    classes, scores, boxes = model.detect(frame, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    for (classid, score, box) in zip(classes, scores, boxes):
        if classid==0 or classid==56:
            cv2.rectangle(frame, box, color, 2)
        if classid==56:
            chair_list.append(box)
        if classid==0:
            human_count+=1
    #cv2.imshow(frame)
    #plt.show()
    if not human_count:
        continue
    print(classes)
    if human_count<=2:
        for (classid, score, box) in zip(classes, scores, boxes):
            if classid==0:
                #print('loop1')
                y1=box[1]
                y2=box[1]+box[3]
                all_angle=[]
                all_ratio=[]
                max_ratio=0
                x=box[0]
                y=box[1]
                delx=box[2]
                dely=box[3]

                angle=-90
                clip=frame[y:y+dely,x:x+delx]
                #plt.imshow(clip)
                clip1=Image.fromarray(clip)
                frame1=cv2.blur(frame,(75,75),cv2.BORDER_DEFAULT)
                img1 = Image.fromarray(frame1)
                img1.paste(clip1, (x,y))

                frame1=asarray(img1)
                #plt.imshow(frame1)
                while angle <=90:
                    frame2 = ndimage.rotate(frame1, angle, reshape=False)
                    classes, scores, boxes = model.detect(frame2, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)

                    for (classid, score, boxx) in zip(classes, scores, boxes):
                        x11=boxx[0]
                        y22=boxx[1]
                        delx1=boxx[2]
                        dely1=boxx[3]
                        ratio=dely1/delx1
        #                 reading=[ratio,angle]
        #                 print(reading)
                        all_ratio.append(ratio)
                        all_angle.append(angle)
                        print(ratio,angle)
                        cv2.rectangle(frame2, boxx, color, 2)
                        break
        #             fig.add_subplot(row,col,i)

        #             cv2.cvtColor(frame1,cv2.COLOR_RGB2BGR)
        #             plt.imshow(frame2)
        #             plt.axis('off')
        #             plt.title(str(angle))
                    i=i+1
                    angle=angle+step

                for (rat,ang) in zip(all_ratio,all_angle):
                    if rat>max_ratio:
                        max_ratio=rat
                        max_angle=ang
                print("Maximum ratio found was ",max_ratio,"at angle",max_angle)
                flag=0
                if max_angle<=10 and max_angle>=-10:
                    box1=box
                    for chair in chair_list:

                        box2=chair
        #                 print(box1)
        #                 print(box2)
                        a1,a2,del_a1,del_a2=box1
                        b1,b2,del_b1,del_b2=box2
                        if not (b1+del_b1<a1 or b1>a1+del_a1 or b2>a2+del_a2 or b2+del_b2<a2):
        #                     print("flag1")
                            print(box1)
                            print(box2)

                            X1=max(a1,b1)
                            X2=min(a1+del_a1,b1+del_b1)
                            Y1=max(a2,b2)
                            Y2=min(a2+del_a2,b2+del_b2)
        #                     if (X2-X1)<=0 or (Y2-Y1)<=0:
        #                         continue

                            common_area=(X2-X1)*(Y2-Y1)
                            print(common_area)
        #                     print(X1,X2,Y1,Y2)
        #                     print("X2-X1",X2-X1,"Y2-Y1",Y2-Y1)
                            human_area=del_a1*del_a2
                            print(human_area)
                            common_per=(common_area/human_area)*100
                            if common_per>0:
                                print(common_per)
                            if(common_per>20):
                                decision="Sitting"
                                flag=1
                                break
                        if not flag:
                            continue
                    if not flag: 
                        print("y1,y2: ",y1,y2)
                    if(y1<460):
                        flag=1
                        decision="Standing"
                        print(decision)
                        if human_count:
                            for (classid, score, box) in zip(classes, scores, boxes):
                                if classid==0:
                                    cv2.rectangle(frame, box, color, 2)
                                if human_count>2:
                                    cv2.putText(frame,"Humans detected:"+str(human_count)+" Fall detection inactive", (5, 100), cv2.FONT_HERSHEY_SIMPLEX,2, color, 2)
                                else:
                                    cv2.putText(frame,"Humans detected:"+str(human_count)+" Fall detection active", (5, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 2)
                                cv2.putText(frame, decision, (5, 30), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 2)
                                
                        #count+=1
                        resize_scale=2
                        h,w,_=frame.shape
                        h=round(h/resize_scale)
                        w=round(w/resize_scale)
                        resized = cv2.resize(frame, (w,h), interpolation = cv2.INTER_AREA)
                        if human_count:
                            cv2.imshow('frame',resized)
                            cv2.waitKey(1000)
                        continue
#                         if flag:
#                             continue
                    if not flag:
                        l=0
                        #area=0
                        for y0 in range(1,1080,1):  
                            if y0>y1 and y0<y2:
                                yo=1080-y0
                                delta_y=6.42186370842406*(10**(-13))*(yo**5) - 8.99277202448275*(10**(-10))*(yo**4) + 4.81220035124696*(10**(-7))*(yo**3) - 0.00011090378397514*(yo**2) + 0.0114193928178952*yo + 0.0586631040713231

                                l+=delta_y
                                #del_area=delta_y*delta_x*delx
                                #area+=del_area
                                print("yo: ",yo," l: ",l)
                        print("length: ",l)
                        if l<320:
                            decision="Fallen"
                        else: decision="Standing"
                if max_angle>10 or max_angle<-10:
                    box1=box
                    flag1=0
                    for chair in chair_list:

                        box2=chair
                        a1,a2,del_a1,del_a2=box1
                        b1,b2,del_b1,del_b2=box2
                        if not (b1+del_b1<a1 or b1>a1+del_a1 or b2>a2+del_a2 or b2+del_b2<a2):
                            X1=max(a1,b1)
                            X2=min(a1+del_a1,b1+del_b1)
                            Y1=max(a2,b2)
                            Y2=min(a2+del_a2,b2+del_b2)
        #                     if (X2-X1)<=0 or (Y2-Y1)<=0:
        #                         continue
        #                     print(box1)
        #                     print(box2)
                            common_area=(X2-X1)*(Y2-Y1)
        #                     print(X1,X2,Y1,Y2)
        #                     print("X2-X1",X2-X1,"Y2-Y1",Y2-Y1)
                            human_area=del_a1*del_a2
                            common_per=(common_area/human_area)*100
                            if common_per>0:
                                print(common_per)
                            if(common_per>20):
                                decision="Sitting"
                                flag1=1
                                break
                    if not flag1:
                        decision="Fallen"
                print(decision)
#     if decision=="Fallen":
#         playsound(r'C:\Users\aksha\Downloads\alarm_tone.mp3')
                
        
    if human_count:
        for (classid, score, box) in zip(classes, scores, boxes):
            if classid==0:
                cv2.rectangle(frame, box, color, 2)
            if human_count>2:
                cv2.putText(frame,"Humans detected:"+str(human_count)+" Fall detection inactive", (5, 100), cv2.FONT_HERSHEY_SIMPLEX,2, color, 2)
            else:
                cv2.putText(frame,"Humans detected:"+str(human_count)+" Fall detection active", (5, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 2)
            cv2.putText(frame, decision, (5, 30), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 2)
            
    count+=1
    resize_scale=2
    h,w,_=frame.shape
    h=round(h/resize_scale)
    w=round(w/resize_scale)
    resized = cv2.resize(frame, (w,h), interpolation = cv2.INTER_AREA)
    if human_count:
        cv2.imshow('frame',resized)
        cv2.waitKey(1000)

[ 0 56 56 56 56 60 62 62 62 62 62]
0.42338709677419356 -90
0.5495867768595041 -80
0.8062015503875969 -60
1.903225806451613 -10
2.295918367346939 0
1.9912280701754386 10
0.5287356321839081 80
Maximum ratio found was  2.295918367346939 at angle 0
y1,y2:  433 666
Standing
[ 0 56 56 56 56 60 62 62 62 62 62]
0.42338709677419356 -90
0.5495867768595041 -80
0.8062015503875969 -60
1.903225806451613 -10
2.295918367346939 0
1.9912280701754386 10
0.5287356321839081 80
Maximum ratio found was  2.295918367346939 at angle 0
y1,y2:  433 666
Standing
[ 0 56 56 56 56 60 62 62 62 62 62]
1.2575757575757576 -50
1.7232142857142858 -40
1.5819672131147542 -30
1.7941176470588236 -20
1.5478260869565217 -10
1.7604166666666667 0
1.6071428571428572 10
1.1568627450980393 20
0.7411764705882353 70
Maximum ratio found was  1.7941176470588236 at angle -20
Fallen
[ 0 56 56 56 62 62 62 62]
0.4849624060150376 -90
1.9694656488549618 -20
1.9492753623188406 -10
1.8985507246376812 0
1.8620689655172413 10
0.5098666666666667 50

In [2]:
pip install playsound==1.2.2

  Attempting uninstall: playsound
    Found existing installation: playsound 1.3.0
    Uninstalling playsound-1.3.0:
      Successfully uninstalled playsound-1.3.0
Note: you may need to restart the kernel to use updated packages.


In [2]:

#playsound(r'C:\Users\aksha\Downloads\alarm_tone.mp3')
import multiprocessing
import time
from playsound import playsound
# def siren():
#     print("Flag")
#     print("playing")
#     for i in range(1,100):
#         print("playing")
#         playsound("C:/Users/aksha/Downloads/alarm_tone.mp3")
#         time.delay(1)
# if __name__ == '__main__':
    
p = multiprocessing.Process(target=playsound, args=("C:/Users/aksha/Downloads/alarm_tone.mp3",))



p.start()
#     while True:
#         print("Fallen")
input("press ENTER to stop playback")
p.terminate()

press ENTER to stop playback


In [1]:
import multiprocessing
import time
from playsound import playsound
p = multiprocessing.Process(target=playsound, args=(r'C:/Users/aksha/Downloads/alarm_tone.mp3',))
p.start()
input("press ENTER to stop playback")
p.terminate()

press ENTER to stop playback


In [3]:
try: 
    while True:
        print("Fallen")
        time.sleep(1)
except KeyboardInterrupt:
    print("Press Ctrl-C to terminate while statement")
    pass
print("loop terminated")

Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Fallen
Press Ctrl-C to terminate while statement
loop terminated
